In [3]:
import os
import pandas as pd
import statsmodels
from sklearn import preprocessing, linear_model, model_selection, metrics

import matplotlib
import matplotlib.pyplot as plt

pd.set_option('display.max_rows',10)
pd.set_option('display.max_columns',10)
pd.set_option('display.notebook_repr_html',True)
%matplotlib notebook

matplotlib.style.use('ggplot')

df = pd.read_csv(os.path.join('..',"data","consumer_complaints.csv.gz"))



//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (5,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df

,date_received,product,sub_product,issue,sub_issue,...,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed?,complaint_id
0,08/30/2013,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,...,09/03/2013,Closed with explanation,Yes,Yes,511074
1,08/30/2013,Mortgage,Other mortgage,"Loan servicing, payments, escrow account",NaN,...,09/03/2013,Closed with explanation,Yes,Yes,511080
2,08/30/2013,Credit reporting,NaN,Incorrect information on credit report,Account status,...,09/18/2013,Closed with explanation,Yes,No,510473
3,08/30/2013,Student loan,Non-federal student loan,Repaying your loan,Repaying your loan,...,08/30/2013,Closed with explanation,Yes,Yes,510326
4,08/30/2013,Debt collection,Credit card,False statements or representation,Attempted to collect wrong amount,...,08/30/2013,Closed with explanation,Yes,Yes,511067
...,...,...,...,...,...,...,...,...,...,...,...
555952,07/01/2014,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,...,07/07/2014,Closed with explanation,Yes,No,919529
555953,07/01/2014,Mortgage,Other mortgage,"Loan servicing, payments, escrow account",NaN,...,07/23/2014,Closed with explanation,No,No,918447
555954,07/10/2012,Mortgage,Conventional fixed mortgage,"Loan modification,collection,foreclosure",NaN,...,11/18/2013,Closed with explanation,Yes,No,114550
555955,04/14/2015,Debt collection,I do not know,Communication tactics,Frequent or repeated calls,...,04/14/2015,Untimely response,No,No,1329963


In [ ]:
df_datetime = pd.to_datetime(df['date_received'])

In [ ]:
df_datetime.head()

In [ ]:
products_df = df[ ['product', 'complaint_id'] ].groupby('product').count().rename(columns = {'complaint_id': 'count'}).reset_index()

### Mortgages are product complained about most

In [ ]:
products_df = products_df.sort_values(by = 'count', ascending = False)

In [ ]:
products_df.plot(x = 'product', y ='count', kind = 'bar', alpha = .75, rot = 90)
plt.tight_layout()


In [ ]:
df_mortgage_issues = df[df['product'] == 'Mortgage'][['issue','complaint_id']].groupby('issue').count().rename(columns = {'complaint_id': 'count'}).reset_index()

In [ ]:
df_mortgage_issues = df_mortgage_issues.sort_values(by = 'count', ascending = False)

### The most common issues are foreclosure & mortgage broker complaints

In [ ]:
df_mortgage_issues.plot(x='issue', y= 'count', kind = 'bar', alpha=0.75, rot=90)

plt.tight_layout()


In [ ]:
df_mortgage_issues_NY = df[df['product'] == 'Mortgage'][df['state'] == 'NY'][['issue','complaint_id']].groupby('issue').count().reset_index()

# New York Issues 

In [ ]:
#these look similar to the national average

df_mortgage_issues_NY.plot(x='issue', y= 'complaint_id', kind = 'bar', alpha=0.75, rot=90)
plt.tight_layout()


In [ ]:
#plot all mortgage complaints responded to in a timely-manner
#compare all mortgage complaints not responded to in a timely manner


#df_mortgage_timely_response = df[df['product'] == 'Mortgage'][df['timely_response'== 'yes']][['issue','complaint_id']].groupby('issue').count().reset_index()

In [ ]:
df_product = pd.get_dummies(df['product'])

In [ ]:
df_product.columns

In [2]:
df_product = pd.get_dummies(df, columns=['product', 'consumer_consent_provided', 'timely_response'])

NameError: name 'pd' is not defined

In [17]:
df_product.columns

Index(['date_received', 'sub_issue', 'consumer_complaint_narrative', 'zipcode',
       'tags', 'consumer_consent_provided', 'submitted_via',
       'date_sent_to_company', 'company_response_to_consumer',
       'timely_response',
       ...
       'state_TN', 'state_TX', 'state_UT', 'state_VA', 'state_VI', 'state_VT',
       'state_WA', 'state_WI', 'state_WV', 'state_WY'],
      dtype='object', length=3841)

In [ ]:
#i want to know what issues people in new york have. are there more foreclosures? 
#foreclosure top 10 : delaware, new jersey, maryland, illinois, south carolina, nevada, florida, ohio, pennsylvania, georgia 



In [ ]:
df_with_expanded_product = pd.get_dummies(df, columns=['product'])

In [ ]:
df_with_expanded_product.columns

## Complaints by state

In [1]:
df_mortgage_counts_state = df_with_expanded_product.groupby('state').sum()

NameError: name 'df_with_expanded_product' is not defined

In [ ]:
df_mortgage_counts_state

In [ ]:
# read in census data 

df_pop_state = pd.read_csv(os.path.join('..',"data","state_pop_abbreviations.csv"))


In [ ]:
df_pop_state.head()



In [ ]:
df_pop_state

In [ ]:
df_mortgage_by_counts_state = df_with_expanded_product.groupby(['state']).sum()

In [ ]:
df_mortgage_by_counts_state

In [ ]:
#create a series with summed number of complaints by state

df_mortgage_state_only = df_mortgage_by_counts_state['product_Mortgage']

In [ ]:
type(df_mortgage_state_only)

In [ ]:
df_mortgage_plot = df_mortgage_state_only.to_frame()

In [ ]:
df_mortgage_plot.plot(kind = 'bar')

In [ ]:
#turn object into dataframe
df_mortgage_state_complaints = df_mortgage_state_only.to_frame().reset_index()

In [ ]:
df_mortgage_state_complaints

In [ ]:
df_mortgage_state_complaints.state.unique()

In [ ]:
df_pop_state.State_.unique()

In [ ]:
df_mortgage_state_complaints = df_mortgage_state_complaints.merge(right = df_pop_state, how = 'inner', left_on = 'state', right_on = 'State_')

In [ ]:
df_mortgage_state_complaints.Pop = df_mortgage_state_complaints.Pop.apply(lambda x: int(x.replace(',', '')))

In [ ]:
df_mortgage_state_complaints

In [ ]:
df_mortgage_state_complaints

In [ ]:
#create a new column that has the complaints per capita 
#complaints per capita == num of complaints/ population 

In [ ]:
df_mortgage_state_complaints.head()




In [ ]:
df_mortgage_state_complaints['mortgage_complaints_per_capita'] = df_mortgage_state_complaints['product_Mortgage']/ df_mortgage_state_complaints['Pop']


In [ ]:
df_mortgage_state_complaints.head()

In [ ]:
state_mortgage_complaintspc = df_mortgage_state_complaints[['mortgage_complaints_per_capita', 'state']]


In [ ]:
state_mortgage_complaintspc.set_index('state', inplace = True)


In [ ]:
state_mortgage_complaintspc.plot(kind='bar')

In [ ]:
df_mortgage_by_counts_state['product_Mortgage'].plot(kind = "bar")

In [ ]:
df_counts_by_state = df_with_expanded_product.groupby(['state']).count()

In [ ]:
df_counts_by_state

In [ ]:
df_mortgage_by_counts_state['product_Payday loan'].plot(kind = "bar")

In [ ]:
df_mortgage_by_counts_state['product_Credit card'].plot(kind = "bar")

## Group by Product 

In [ ]:
#this is creating a new column that is summing something unknown ???!!!
#df_product['sum_of_complaints_by_product'] = df_product.sum(axis=1)

In [ ]:
df_product

In [ ]:
df_with_expanded_product.columns

In [ ]:

df_issue = df_with_expanded_product.groupby(['issue']).plot()

In [ ]:
df_issue

In [ ]:
consumer_complaint_narrative = df_with_expanded_product[ ['issue', 'consumer_complaint_narrative'] ].groupby('product_Mortgage').count().rename(columns = {'complaint_id': 'count'}).reset_index()

## Predict who complains based on the type of product 

In [ ]:
X = df_with_expanded_product[ ['state'] ]
y = df_with_expanded_product.product_Mortgage

In [ ]:
type(X)

In [ ]:
model = linear_model.LinearRegression()
model.fit(X,y)

print (model.intercept_)
print (model.coef_)

In [ ]:
logistic regression -- time and state 
    

In [ ]:
df_with_expanded_product.head()

# Random Forest 

In [ ]:
from sklearn import tree, ensemble, metrics, model_selection, externals

In [ ]:
df_with_expanded_product.head()

In [ ]:
df_with_expanded_product.columns

In [ ]:
X = df_with_expanded_product[df_with_expanded_product.columns.values]
X.drop('product_Mortgage', axis = 1, inplace = True)

y = df_with_expanded_product.product_Mortgage

In [ ]:
train_X, test_X, train_y, test_y = model_selection.train_test_split(X, y, train_size = .6, random_state = 0)


In [ ]:
model = tree.DecisionTreeRegressor(random_state = 0).fit(train_X, train_y)


In [ ]:
train_y_hat = model.predict(train_X)
print (np.sqrt(metrics.mean_squared_error(train_y, train_y_hat)))

test_y_hat = model.predict(test_X)
print (np.sqrt(metrics.mean_squared_error(test_y, test_y_hat)))

In [ ]:
dot_data = externals.six.StringIO()
tree.export_graphviz(model, out_file = dot_data, feature_names = X.columns)
%dotstr dot_data.getvalue()